# 60_days_udacity: which day am I on? 

## Motivation

As I started with the 60_days_udacity challenge, I never could recall which day of the challenge I am on (is it 11th or 12th?). At this point I already finished the 'github' part and started with the 'Cloud Fundamentals' part. Thus, I decided to use the knowledge acquired to

    * calculate the day with the AWS Lambda (with python as Runtime)
    * trigger Lambda with AWS CloudWatch event
    * deliver message to my e-mail with AWS SNS
    
I consider this small project as an easy and at the same time interesting interplay of 3 AWS services mentioned above.
    
By the way, this was also the start of my journey with boto3 - the AWS SDK for python.

## SNS Topic to sent notifications

I start with the SNS service to deliver messages to my mailbox. At lesson 17 we created SNS topic, subscribed to it and published messages, correct? My topic was called 'us_west_2_notifications'. I hope you created your topic too. I decided to use the same topic for my messages. 

Go to 'Simple Notification Service' -> 'Topics', your topic should be there. Click on it and pay attention to 'Details'. There you find the ARN, which will be required in your Lambda function. That's it.

<img src="pictures/SNS-topic-Details.png" alt="SNS Topic ARN" width="400"/>

## Lambda function

Go to 'Lambda' -> 'Functions' and create the Lambda function as we learned in Lesson 13. Select `Python 3.8` as 'Runtime'.


The code of the function is given below. It just subtracts my starting day from the current day

In [1]:
import boto3

def lambda_handler(event, context):
    today = datetime.today()
    difference = today - datetime(2019, 12, 10)
    
    message = {"60_days_udacity": "I am on a day {:d}".format(difference.days)}
    client = boto3.client('sns')
    response = client.publish(
        TargetArn="arn:aws:sns:us-west-2:342028678820:us_west_2_notifications",
        Message=json.dumps({'default': json.dumps(message)}),
        MessageStructure='json'
    )

    return {
        'statusCode': 200,
        'body': json.dumps('published via sns topic')
    }


Let us go through the code.

The actual calculation of the day occcurs on lines 4 and 5.
Line 7 creates a text message that will be sent to my smartphone.

On line 8 I am using the boto3 library (which was imported on line 1) to create an sns client called simply 'client' in the code. 

To publish the response with SNS topic you need to know its ARN. This is exactly what we got in the previous section.

Lines 11 and 12 are boilerplate code to enforce the json format of the message.

Lines 15 to 18 are boilerplate code to return the OK status in 'statusCode' and the result of calculations (number of days from my starting date) in 'body'.


After you enter the code, the Lambda function should look like this

<img src="pictures/lambda-python.png" alt="AWS Lambda with Python runtime" width="700"/>

##  CloudWatch Events to trigger the Lambda function

Go to 'CloudWatch' service and create the CloudWatchEvent as we learned in Lesson 18. Choose `Schedule` as Event Source. 
To schedule the events at 0 minutes 9 hours UTC time every day, set the 'Cron Expression' to `0 9 ? * * *`. If this expression looks misterious to you, have a look at the [Schedule Expressions for Rules](https://docs.aws.amazon.com/AmazonCloudWatch/latest/events/ScheduledEvents.html) or a nice explanation at [Cron Format](http://www.nncron.ru/help/EN/working/cron-format.htm). 

<img src="pictures/CloudWatch-cron.png" alt="CloudWatch CRON Schedule" width="700"/>


Set Target to 'Lambda function' and from the 'Function' drop-down menu select the name of the function you created. In my case - `dayOnUdacityPython`.

<img src="pictures/CloudWatch-targets.png" alt="CloudWatch Targets" width="700"/>

After your rule is created, click on it to get the Summary. This is how it looks for my function

<img src="pictures/CloudWatch-summary.png" alt="CloudWatch Summary" width="500"/>

For more details on creating such rules see [Creating a CloudWatch Events Rule That Triggers on a Schedule](https://docs.aws.amazon.com/AmazonCloudWatch/latest/events/Create-CloudWatch-Events-Scheduled-Rule.html)

## Add trigger to your Lambda function

Now it is time to a add a trigger to your function. Go back to 'Amazon Lambda' service and select your function.
Click on 'Add trigger' and choose `CloudWatchEvents`.
At the 'Rule' select the rule, that you created at CloudWatch. In my case it is `dayOnUdacityChallenge`.

<img src="pictures/lambda-addTrigger.png" alt="Lambda add Trigger" width="600"/>

After your function is set, it should look like

<img src="pictures/lambda-CloudWatchAsTrigger.png" alt="CloudWatch as Trigger" width="700"/>


You are done. Each day at 9 UTC you will get the e-mail message from Amazon.

## Did you complete your 60_days_udacity challenge?

Then its time to disable your rule. Go to 'CloudWatch' -> 'Rules', choose your rule by clicking on the bullet in the 'Status' column and choose 'Actions' -> `Disable` or `Delete`. As I completed the challenge, I only disabled my rule. I also kept my Lambda function and SNS topic. That is how I am able to write this post now.